In [2]:
from pydantic import BaseModel, Field, model_validator
from typing import List, TypedDict
import re
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
import os
from serpapi import GoogleSearch
from langchain.tools import tool
from langchain import hub

client = OpenAI()
# # Skeleton

# 1. Text input
# 2. ExtractClaims (text: string)
# 3. Plan verification ()
# 4. Execute verification
# 5. Return result

class Claim(BaseModel):
    claim: str = Field(...)
    substring_quote: List[str] = Field(...)

    @model_validator(mode="after")
    def validate_sources(self) -> "Claim":
        # Assuming text_chunk is provided during initialization
        text_chunks = getattr(self, "text_chunk", None)
        if text_chunks:
            spans = list(self.get_spans(text_chunks))
            self.substring_quote = [text_chunks[span[0]:span[1]] for span in spans]
        return self

    def get_spans(self, context):
        for quote in self.substring_quote:
            yield from self._get_span(quote, context)

    def _get_span(self, quote, context):
        for match in re.finditer(re.escape(quote), context):
            yield match.span()

class Claims(BaseModel):
    claims: List[Claim] = Field(description="The claims extracted from the text")

class VerificationStep(BaseModel):
    step_to_verify: str = Field(description="The step to verify the claim")

class VerificationPlan(BaseModel):
    plan: List[VerificationStep] = Field(description="The plan to verify the claims")

class VerificationResult(BaseModel):
    truthfullness_score: int = Field(description="The truthfullness score of the claim as a number between 1 and 10.")
    sources: List[str] = Field(description="The url sources used to validate or nulify the claim.")
    explanation: str = Field(description="The explanation of the score given to the claim.")

class OutputVerificationPlan(TypedDict):
    claim: str
    truthfullness_score: float 
    sources: List[str]
    explanation: str

def claim_extractor(text: str) -> Claims:
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research, and you extract claims from the text"},
                  {"role": "user", "content": text}],
        response_format=Claims
    )
    
    return response.choices[0].message.parsed

def generate_verification_plan(claim: Claim) -> VerificationPlan:
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful research, and you generate a verification plan for the claim."},
                  {"role": "user", "content": claim.claim}],
        response_format=VerificationPlan
    )
    return response.choices[0].message.parsed

# def execute_verification_plan(plan: VerificationPlan) -> OutputVerificationPlan:
#     llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
#     tools = []

@tool
def web_search(query: str) -> str:
    """
    Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query.
    """
    serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}
    search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
    try:
        results = search.get_dict()["organic_results"]
        print("RESULTS:")
        print(results)
    except Exception as e:
        print(e)
        results = []
        return "No results found"
    
    try:    
        contexts = "\n---\n".join(
             ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
        )
    except Exception as e:
        print(e)
        return "No results found"
    
    return contexts

def execute_verification_plan(claim: str, plan: VerificationPlan) -> OutputVerificationPlan:
    llm = ChatOpenAI(model="gpt-4o-mini")
    tools = [web_search]
    prompt = hub.pull("hwchase17/openai-tools-agent")
    agent = create_tool_calling_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    research_output = ""
    for step in plan.plan:
        result = agent_executor.invoke({
            "input": f"Verify this claim: {claim} by executing this step: {step.step_to_verify}",
            "agent_scratchpad": []
        })
        research_output += result["output"]
        research_output += "\n---\n"
    
    # Parse the final analysis into the required format
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "Parse the analysis from the research made into these fields: truthfullness_score, sources, justification"},
                 {"role": "user", "content": research_output}],
        response_format=VerificationResult
    )
    result = response.choices[0].message.parsed 
    
    return OutputVerificationPlan(
        claim=claim,
        truthfullness_score=result.truthfullness_score,
        sources=result.sources,
        explanation=result.explanation
    )



def process_claims(text: str) -> List[OutputVerificationPlan]:
    # Extract claims from text
    claims = claim_extractor(text)
    
    # Generate verification plan
    for claim in claims.claims:
        verification_plan = generate_verification_plan(claim)
        verification_plans.append(verification_plan)
    
    # Execute verification plan for each claim
    results = []
    for claim, plan in zip(claims.claims, verification_plans):
        result = execute_verification_plan(VerificationPlan(plan=[plan]))
        results.append(result)
        
    return results


# expression_with_statement = """
# The world war 2 was started by the US, and the US won the war.
# """

# claim = claim_extractor(expression_with_statement).claims[0]
# plan = generate_verification_plan(claim)
# research_outputs = execute_verification_plan(claim, plan)
# print(research_outputs)

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [web_search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)
research_output = ""

claim = "The world war 2 was started by the US, and the US won the war."
actionable_step = "Research resources for the US involvement in the world war 2 and output the sources."
result = agent_executor.invoke({
    "input": f"Verify this claim: {claim}, by executing this step: {actionable_step}?",
    "agent_scratchpad": []
})
# research_output += result["output"]
# research_output += "\n---\n"

result



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'US involvement in World War 2 history'}`


RESULTS:
[{'position': 1, 'title': 'Military history of the United States during World War II', 'link': 'https://en.wikipedia.org/wiki/Military_history_of_the_United_States_during_World_War_II', 'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Military_history_of_the_United_States_during_World_War_II&ved=2ahUKEwj407zVt-GJAxX_IzQIHW4tCUwQFnoECDQQAQ', 'displayed_link': 'https://en.wikipedia.org › wiki › Military_history_of_th...', 'favicon': 'https://serpapi.com/searches/6738de13a775f29cd7166686/images/d9b0f4539c0334f0d6332adc0fde484bf67b7333b0e2a35323c19fa8093a8d5d.webp', 'snippet': 'While officially neutral, the US supplied Britain, the Soviet Union, and China with war materiel through the Lend-Lease Act signed into law on 11 March 1941, ...', 'snippet_highlighted_words': ['US supplied Britain, the S

{'input': 'Verify this claim: The world war 2 was started by the US, and the US won the war., by executing this step: Research resources for the US involvement in the world war 2 and output the sources.?',
 'agent_scratchpad': [],
 'output': "Here are some resources that discuss the involvement of the United States in World War II:\n\n1. **Military history of the United States during World War II** - This article provides an overview of the U.S. military history during the war, including its initial neutrality and later involvement.\n   - [Read more here](https://en.wikipedia.org/wiki/Military_history_of_the_United_States_during_World_War_II)\n\n2. **Take A Closer Look: America Goes to War** - This resource discusses the preparations and volunteer efforts of Americans before and during the war.\n   - [Read more here](https://www.nationalww2museum.org/students-teachers/student-resources/research-starters/america-goes-war-take-closer-look)\n\n3. **Lend-Lease and Military Aid to the Allie

In [22]:
result['intermediate_steps'][0][0].message_log[0].response_metadata

{'finish_reason': 'tool_calls',
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_0ba0d124f1'}